In [ ]:
%matplotlib widget

import sys
import warnings
from io import BytesIO
from io import TextIOWrapper
from matplotlib import pyplot as plt
from ipywidgets import interact
from ipywidgets import interactive
from ipywidgets import FileUpload
from ipywidgets import Text
from ipywidgets import Label
from ipywidgets import FloatRangeSlider
from ipywidgets import FloatSlider
from ipywidgets import Tab
from ipywidgets import HBox
from ipywidgets import VBox
from ipywidgets import Layout
from ipywidgets import Button
from ipywidgets import Checkbox

from flowcorrection import Analysis
from shot import Shot

ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    def ignore(x, y):
        return []
    return ipython._showtraceback(etype, value, ignore(etype, value))

ipython.showtraceback = hide_traceback


def make_interactive(a: Analysis):
    def update_flow(correction_val):
        a._update_flow(a.shot.flow, correction_val)
    def update_window(window_value):
        a._update_window(window_value)

    # range slider (optimization window)
    end_time = a.shot.elapsed[-1]
    opt_window_slider = FloatRangeSlider(value=a._initial_window, min=0, max=end_time, step=0.1, readout=False, description='opt. window', layout=Layout(width='90%'))
    interact(update_window, window_value=opt_window_slider)

    # vertical slider, adjustment buttons (correction value)
    correction_slider = FloatSlider(value=1.0, min=0.3, max=2.5, step=0.01, readout=False, description='correction value', layout=Layout(width='90%'))
    minus_btn = Button(description='< (-0.01)')
    plus_btn = Button(description='(+0.01) >')
    # buttons
    def adjust_correction(offset: float):
        correction_slider.value += offset
    minus_btn.on_click(lambda _: adjust_correction(-0.01))
    plus_btn.on_click(lambda _: adjust_correction(0.01))
    # value box
    correction_val = Text(value='', description='', disabled=True, layout=Layout(width='15%'))
    def set_correction_text(_=None):
        if a.shot.current_calibration == 1.0:
            value = 'x%.02f' % correction_slider.value
            correction_val.value = value
        else:
            value = 'x%.03f\n(%.02f)' % (a.shot.current_calibration * correction_slider.value, correction_slider.value)
            correction_val.value = value
    correction_slider.observe(set_correction_text, names='value')
    set_correction_text()

    correction_bottom = HBox([minus_btn, plus_btn, correction_val])
    correction_box = VBox([interactive(update_flow, correction_val=correction_slider), correction_bottom], layout=Layout(width='100%'))
    display(correction_box)


def from_url(visualizer_url: str, verbose: bool):
    try:
        if not visualizer_url.startswith('http://') and not visualizer_url.startswith('https://'):
            raise RuntimeError('invalid shot url!')
        s = Shot.parse_visualizer(visualizer_url)
        a = Analysis(s)
        a.show(False, verbose)
        make_interactive(a)
        note1 = Label(value='IMPORTANT! Current calibration factor is unknown!')
        note2 = Label(value='Multiply the current calibration factor by this correction value!')
        display(HBox([note1, note2]))
    except Exception as e:
        display(Label(value='ERROR: %s' % e))


def from_file(uploaded, verbose: bool):
    try:
        for name in uploaded:
            raw_shot_buf = BytesIO(uploaded[name]['content'])
            raw_shot_buf.name = name
            shot_input = TextIOWrapper(raw_shot_buf, encoding='utf-8')
            s = Shot.parse(shot_input)
            a = Analysis(s)
            a.show(False, verbose)
            make_interactive(a)
            break
    except Exception as e:
        display(Label(value='ERROR: %s' % e, color='red'))

warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (9, 6)

In [ ]:
# Parameters:
verbose = False
shot_url = None


In [ ]:
if shot_url:
    from_url(shot_url, verbose)
else:
    tab = Tab()
    for idx, s in enumerate(['from .shot file', 'from visualizer.coffee']):
        tab.set_title(idx, s)
    tab_children = [
        interactive(from_file,
                    uploaded=FileUpload(accept='.shot', multiple=False),
                    verbose=Checkbox(description='show verbose graph!').value),
        interactive(from_url, {'manual': True, 'manual_name': 'Run'},
                    visualizer_url=Text('https://visualizer.coffee/shots/...').value,
                    verbose=Checkbox(description='show verbose graph!').value)
    ]
    tab.children = [VBox(children=(tab_children[0],)), VBox(children=(tab_children[1],))]
    display(tab)